In [1]:
import numpy as np
import cv2

In [4]:
P1 = np.array([[-6.5920441e+01,  8.5485431e+02, -4.0555966e+02,  1.2354684e+03],
       [ 5.9958112e+02,  4.3894717e+02,  2.5966882e+02,  1.1042605e+03],
       [ 5.5738282e-01,  4.6548709e-01, -6.8749267e-01,  1.9879433e+00]], dtype=np.float32)

P2 = np.array([[-8.0220477e+02, -4.8673218e+01, -5.0368481e+02,  1.2422305e+03],
       [-4.5547061e+02,  6.0689667e+02,  2.0933946e+02,  1.1396505e+03],
       [-3.7105042e-01,  5.6809044e-01, -7.3457122e-01,  1.9341261e+00]], dtype=np.float32)

hP = cv2.triangulatePoints(P1, P2, np.array([867, 454], dtype=np.float32), np.array([778, 673], dtype=np.float32))
hP /= hP[3]

print(hP)

[[-0.71007943]
 [ 0.21607947]
 [-0.02937308]
 [ 1.        ]]


In [3]:
WIDTH = 1280
HEIGHT = 720

focal_length = 2.1
#focal_length = 3.6651 # calculated from sensor width and fov
principal_point = (WIDTH//2, HEIGHT//2)
print(principal_point)

camera_matrix = np.array([
    [focal_length/0.003, 0, 640],
        [0, focal_length/0.003, 360],
        [0, 0, 1]
    ], dtype=np.float32)

dist_coeffs = np.array([1.40017137e-14, 2.14617695e-10, -1.38004061e-16, 2.35596742e-16, -6.64757209e-15], dtype=np.float32)

(640, 360)


In [4]:
camera0_points = np.array([(803, 604)], dtype=np.float32)
camera2_points = np.array([(481, 447)], dtype=np.float32)
camera4_points = np.array([(459, 596)], dtype=np.float32)

R0 = np.array([
    [
      0.9291198583657363,
      -0.36977870245652894,
      2.1083672978278807e-16
    ],
    [
      -0.1687457196047532,
      -0.42399683393725024,
      -0.8898042295506703
    ],
    [
      0.32903065344357835,
      0.8267347797333519,
      -0.45634245153583747
    ]
  ],
    dtype=np.float32)

R2 = np.array([
      [
         0.9187921918931989,
         -0.39474157130468485,
         6.171606320936834e-17
      ],
      [
         -0.18013733637228838,
         -0.4192838813005277,
         -0.8898042295506704
      ],
      [
         0.3512427197263861,
         0.8175451784246993,
         -0.4563424515358372
      ]
   ], 
    dtype=np.float32)

R4 = np.array([
      [
         -0.9841162324872018,
         -0.17752532482774327,
         -3.9522300551188685e-16
      ],
      [
         0.08101234194158781,
         -0.4490940141294215,
         0.8898042295506704
      ],
      [
         -0.1579627848840828,
         0.875670786036583,
         0.45634245153583686
      ]
   ], 
    dtype=np.float32)

t0 = np.array([
    [
      0.07841704591746641
    ],
    [
      -0.9000390651104047
    ],
    [
      1.7549508361555015
    ]
  ], dtype=np.float32)
t2 = np.array([
      [
         0.9955215054763127
      ],
      [
         -1.5964170431688387
      ],
      [
         3.1127909147125523
      ]
   ], dtype=np.float32)
t4 = np.array([
      [
         -0.36042305026837873
      ],
      [
         -0.6178865081570158
      ],
      [
         1.2047926430908236
      ]
   ], dtype=np.float32)

Rt0 = np.hstack((R0, t0))
Rt2 = np.hstack((R2, t2))
Rt4 = np.hstack((R4, t4))

P0 = camera_matrix @ Rt0
P2 = camera_matrix @ Rt2
P4 = camera_matrix @ Rt4

In [5]:
def get_fundamental_matrix(K1, K2, R1, R2, t1, t2):
    R = R2 @ R1.T #relative rotation
    t = t2 - R @ t1 #relative translation


    # Compute skew-symmetric matrix of t
    t_skew = np.array([
        [0, -t[2, 0], t[1, 0]],
        [t[2, 0], 0, -t[0, 0]],
        [-t[1, 0], t[0, 0], 0]
    ])

    # Compute Essential Matrix
    E = t_skew @ R

    # Compute Fundamental Matrix
    F = np.linalg.inv(K2).T @ E @ np.linalg.inv(K1)

    # Normalize Fundamental Matrix
    F /= np.linalg.norm(F)

    return F

In [6]:
def check_epipolar_constraint(F, p1, p2):

    p1 = np.array([p1[0], p1[1], 1])
    p2 = np.array([p2[0], p2[1], 1])

    print(p1, p2)
    print(p2 @ F @ p1)

    return np.abs(p2 @ F @ p1) < 0.01

In [8]:
foundCorrespondences = []

for point0 in camera0_points:
    for point4 in camera4_points:
        if(check_epipolar_constraint(get_fundamental_matrix(camera_matrix, camera_matrix, R0, R4, t0, t4), point0, point4)):
            foundCorrespondences.append((point0, point4))
#     for point4 in camera4_points:
#         if(check_epipolar_constraint(get_fundamental_matrix(camera_matrix, camera_matrix, R0, R4, t0, t4), point0, point4)):
#             foundCorrespondences.append((point0, point4))

# for point2 in camera2_points:
#     for point4 in camera4_points:
#         if(check_epipolar_constraint(get_fundamental_matrix(camera_matrix, camera_matrix, R2, R4, t2, t4), point2, point4)):
#             foundCorrespondences.append((point2, point4))

[803. 604.   1.] [459. 596.   1.]
-193.7590802195607


In [34]:
triangulated_points = []

for point0, point1 in foundCorrespondences:
    homogeneous_point = cv2.triangulatePoints(P0, P4, point0, point1)
    point = homogeneous_point / homogeneous_point[3]
    triangulated_points.append(point)
    print(point0, point1, point)
    

[673. 489.] [745. 593.] [[-0.3473977 ]
 [-0.05675602]
 [-0.02339257]
 [ 1.        ]]
[748. 595.] [564. 636.] [[-0.02144227]
 [ 0.49490538]
 [-0.02843176]
 [ 1.        ]]
[547. 616.] [513. 531.] [[ 0.4852258 ]
 [ 0.06063693]
 [-0.03111175]
 [ 1.        ]]


In [21]:
homogeneous_point = cv2.triangulatePoints(P0, P4, np.array([[675, 577]]).T, np.array([[617, 647]]).T)
point = homogeneous_point / homogeneous_point[3]
print(point)

error: OpenCV(4.10.0) /io/opencv/modules/calib3d/src/triangulate.cpp:75: error: (-209:Sizes of input arguments do not match) Number of proj points coordinates must be == 2 in function 'icvTriangulatePoints'
